In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from utils import datasets, utils
import os
import time

from methods.covid import ours

In [2]:
x_train, x_test, y_train, y_test, classes = datasets.load('covid')

domains = x_train.keys()

In [3]:
for domain in domains:
    print(domain, 'train', x_train[domain].shape)
    print(np.unique(y_train[domain], return_counts=True))
    print(domain, 'test', x_test[domain].shape)
    print(np.unique(y_test[domain], return_counts=True))
    print()

1 train (800, 17)
(array([0, 1]), array([ 59, 741]))
1 test (2000, 17)
(array([0, 1]), array([ 160, 1840]))

2 train (800, 17)
(array([0, 1]), array([ 99, 701]))
2 test (2000, 17)
(array([0, 1]), array([ 259, 1741]))

3 train (800, 17)
(array([0, 1]), array([110, 690]))
3 test (2000, 17)
(array([0, 1]), array([ 277, 1723]))

4 train (800, 17)
(array([0, 1]), array([ 84, 716]))
4 test (2000, 17)
(array([0, 1]), array([ 216, 1784]))

5 train (800, 17)
(array([0, 1]), array([ 39, 761]))
5 test (2000, 17)
(array([0, 1]), array([  78, 1922]))



## Parameters

In [4]:
domain_t = '1'

lr = 1e-5
batch_size = 50
epochs = 100
m_params = (epochs, batch_size, lr, len(classes))
evaluation_metrics = ['acc', 'bal_acc', 'auc', 'f1']

## Select source domains

In [5]:
x_sources, y_sources = {}, {}
for domain_s in domains:
    if domain_s != domain_t:
        x_sources[domain_s] = x_train[domain_s]
        y_sources[domain_s] = y_train[domain_s]

## Method execution

In [6]:
## Instantiate method
method = ours.Method(*m_params)

## Train method
method.train(x_sources, y_sources, x_train[domain_t], y_train[domain_t], domain_t=domain_t)

## Obtain predictions from trained method on target domain test set
y_probas = method.predict(x_test[domain_t])

## Eval results given evaluation metrics
met = utils.eval_results(y_probas, y_test[domain_t], evaluation_metrics)

Epoch 1/100 - Time elapsed 00:00:01
Epoch 10/100 - Time elapsed 00:00:08
Epoch 20/100 - Time elapsed 00:00:14
Epoch 30/100 - Time elapsed 00:00:20
Epoch 40/100 - Time elapsed 00:00:26
Epoch 50/100 - Time elapsed 00:00:32
Epoch 60/100 - Time elapsed 00:00:38
Epoch 70/100 - Time elapsed 00:00:44
Epoch 80/100 - Time elapsed 00:00:49
Epoch 90/100 - Time elapsed 00:00:55
Epoch 100/100 - Time elapsed 00:01:00


In [8]:
met

{'acc': 84.35000000000001,
 'bal_acc': 88.6413043478261,
 'auc': 94.40743885869564,
 'f1': 87.41325218629335}